In [1]:
import os
import numpy as np
import pandas as pd


In [45]:
%run yelp

In [3]:
rs = load_fc7(directory_out = './train_outputs/')

In [4]:
sample_submission = pd.read_csv('sample_submission.csv', header = 0)

# Number of labels

In [5]:
n_labels = 9

# Classification

In [6]:
train3 = prep_y(pd.read_csv('train.csv', header = 0, index_col = 'business_id'))

<string>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
<string>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


#### Check join

Plan: wrzucamy przyklady jak leci, tj. jesli mam 3 zdjecia danego biznesu - trudno, kazde zdjecie dostanie te sama lebelke.

In [7]:
photo_to_biz_train, df_rs_cleaned_up, photo_prefix = prep_rs(rs)
assert len(set(df_rs_cleaned_up['business_id']) - set(train3['1'].index)) == 0

In [8]:
Xy = df_rs_cleaned_up.merge(train3, left_on = 'business_id', right_index = True)

In [9]:
%clear out

del df_rs_cleaned_up
del rs

In [10]:
#import math
#uniq_business_ids = set(Xy['business_id'].values)

#train_business_ids = uniq_business_ids
#validation_business_ids = set(photo_to_biz_test['business_id'].values)

# train_business_ids, validation_business_ids = train_validation_split(Xy['business_id'].values, 0.8)

In [11]:
#print(len(validation_business_ids))
#print(len(train_business_ids) )

NameError: name 'train_business_ids' is not defined

In [12]:
#train_indices = Xy['business_id'].isin(train_business_ids)
#validation_indices = Xy['business_id'].isin(validation_business_ids)

NameError: name 'train_business_ids' is not defined

In [ ]:
#def featurize():    
#    transformations = [ (i, None) for i in xrange(4096)]
#    return DataFrameMapper(transformations)

In [13]:
#from sklearn_pandas import DataFrameMapper
#from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
clf = OneVsRestClassifier(LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial'))

#pipeline = Pipeline([('featurize', featurize()), ('logistic', clf)])

In [14]:
X_train = Xy.iloc[:,:4096].values

In [15]:
X_train.flags

  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [16]:
max_val_per_feature = X_train.max(axis = 0)

In [21]:
X_train = X_train / max_val_per_feature

In [23]:
X_train = np.ascontiguousarray(X_train)

In [24]:
X_train.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  UPDATEIFCOPY : False

In [22]:
%clear out

In [27]:
y_train = np.ascontiguousarray(Xy.iloc[:,-9:].values)

In [29]:
X_train.nbytes / 1024 / 1024

3664

In [30]:
from scipy.sparse import csr_matrix
X_train_sparse = csr_matrix(X_train)
del X_train

In [103]:
X_train_sparse = csr_matrix(X_train)
del X_train

In [97]:
import datetime
print(datetime.datetime.now())
clf.fit(X_train, y_train)
print(datetime.datetime.now())

2016-02-02 10:34:30.396407
2016-02-02 11:28:22.668777


In [44]:
import cPickle as pickle
with open('clf_lr_for_test', 'wb') as handle:
  pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cPickle as pickle
with open('pipeline_lr_for_test', 'r') as handle:
  pipeline = pickle.load(handle)

#### Use HashingVectorizer?

In [ ]:
labels = ['good_for_lunch','good_for_dinner','takes_reservations','outdoor_seating','restaurant_is_expensive','has_alcohol','has_table_service','ambience_is_classy','good_for_kids']

In [46]:
rs_test = load_fc7(directory_out = './test_outputs/')

In [102]:
rs_test

(237152, 4096)

In [109]:
rs_test = rs_test / max_val_per_feature

In [110]:
np.max(rs_test, axis = 0)

array([ 0.9217099 ,  1.06095362,  1.08061111, ...,  1.18267   ,
        0.78046209,  1.33413303], dtype=float32)

In [111]:

import datetime
print(datetime.datetime.now())

pred_proba = clf.predict_proba(rs_test)
print(datetime.datetime.now())

2016-02-02 12:26:40.556995
2016-02-02 12:29:56.796196


In [112]:
pred = threshold(pred_proba, 0.5)

In [113]:
directory_test = './test_photos/'
photo_ids_test = [int(x[:-4]) for x in sorted(list(os.listdir(directory_test)))]
df_pred = pd.DataFrame(pred, index = photo_ids_test)

In [114]:
#full_pred = pd.concat([Xy.loc[validation_indices]['business_id'], pd.DataFrame(pred, Xy.loc[validation_indices].index)], axis = 1)
photo_to_biz_test = pd.read_csv('test_photo_to_biz.csv', header = 0)
df_pred_with_business_id = df_pred.merge(photo_to_biz_test, left_index = True, right_on = 'photo_id')  
#full_pred = pd.concat([df_rs_test_with_biz['business_id'], pd.DataFrame(pred, index = df_rs_test_with_biz.index)], axis = 1)

In [115]:
result_mean = df_pred_with_business_id.groupby(['business_id']).mean().iloc[:,0:9]

In [116]:
business_pred = threshold(result_mean, 0.5)

In [117]:
result = pd.DataFrame(business_pred.apply(lambda x: np.where(x == 1), axis = 1).apply(lambda x: ' '.join(map(str, x[0]))), 
                      columns = ["labels"])

In [118]:
result.to_csv('submission3_lr_mean.csv')

In [ ]:
np.shape(sample_submission)

In [ ]:
result

In [ ]:
y_valid_business = train3[['1','2','3', '4', '5', '6', '7', '8', '9']].loc[validation_business_ids]
y_valid_business.sort_index(inplace = True)

In [ ]:
assert np.all(y_valid_business.index == business_pred.index)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
#f1_score(y_true = y.iloc[validation_indices,:].values, y_pred = pred)
df_f1 = pd.DataFrame(f1_score(y_true = y_valid_business, y_pred = business_pred, average = None), index = labels)
print(df_f1)
print(pd.DataFrame(precision_score(y_true = y_valid_business, y_pred = business_pred, average = None), index = labels))
print(pd.DataFrame(recall_score(y_true = y_valid_business, y_pred = business_pred, average = None), index = labels))

In [ ]:
print(df_precision)
print(df_recall)

In [ ]:
np.shape(np.where(X == 0))

In [ ]:
np.shape(X)[0] * np.shape(X)[1] - 774289437

In [ ]:
X.values.flags

In [ ]:
df_f1t = (df_f1.transpose())
%load_ext rpy2.ipython
%Rpush df_f1t


In [ ]:
%%R
library(ggplot2)
df_f1


#### TODO uzyj validation set

In [ ]:
business_id = 3037
business_indices = X['business_id'] == business_id
business_X = X.loc[business_indices]
pictures = [directory + photo_prefix[x] + str(x) +".jpg" for x in business_X.index.values.tolist()]

In [ ]:
pred[np.where(business_indices)][2]

In [ ]:
business_labels_true = map(lambda x: labels[int(x)], train.ix[business_id]['labels'].split())

#print(business_labels_pred)
print(business_labels_true)


In [ ]:
picture = pictures[3]
print(picture)
import cv2
%matplotlib inline
from matplotlib import pyplot as plt

img = cv2.cvtColor(cv2.imread(picture), cv2.COLOR_BGR2RGB)
plt.imshow(img)

In [ ]:
# LogisticRegression, PassiveAggressiveClassifier or SGDClassifier
#from sklearn import svm
#clf = svm.LinearSVC()
#clf.fit(X[:100000], y.iloc[:100000,1])

In [ ]:
#clf = svm.SVC(decision_function_shape='ovo')
#clf.fit(X, y.iloc[:,1]) 
#dec = clf.decision_function([[1]])
#dec.shape[1]